# Tutorial 4: Conditional Routing and Decision Making

**Difficulty:** Intermediate | **Time:** 30 minutes

## Learning Objectives

- Create conditional edges between nodes
- Route execution based on node outputs
- Build decision trees with branching logic
- Implement loops and cycles in graphs
- Use multiple routing strategies (lambdas, shorthands, expressions)

## Real-World Use Case

Imagine you're building an AI customer service system that needs to:
- **Route** questions to the right department (sales, support, billing)
- **Decide** whether to use a simple FAQ response or escalate to a human
- **Loop** back to ask clarifying questions if needed
- **Branch** to different workflows based on user sentiment or intent

All of these require **conditional routing** - the ability to dynamically choose the next step in your workflow based on data, outputs, or decisions.

## Core Concepts

### Conditional Routing

Instead of linear flows, you can create **branching** and **cyclic** flows:

```
# Branching
                    ┌→ Node B (if condition 1)
        Node A ─────┤
                    └→ Node C (if condition 2)

# Loops
        Node A → Node B → Node C
           ↑                ↓
           └────────────────┘ (loop back if condition)
```

### Three Ways to Define EdgeConditions

1. **Lambda functions** (most flexible):
   ```python
   node.goto(next_node, condition=EdgeCondition(lambda n: n.outputs.content.get('score') > 80))
   ```

2. **Shorthand equals** (most readable):
   ```python
   node.on(action='search') >> search_node
   ```

3. **Expression strings** (declarative):
   ```python
   node.on(expr="$.outputs.score > 0.8 and $.outputs.ready") >> process_node
   ```

### CRITICAL: Fan-Out vs Single-Path Routing

**Spark uses FAN-OUT routing: ALL matching edges fire simultaneously!**

```python
# When score = 0.8, BOTH edges fire
processor.on(expr="$.outputs.score > 0.7") >> logger
processor.on(expr="$.outputs.score > 0.5") >> validator
```

For **single-path routing**, use **mutually exclusive conditions**:

```python
# Only ONE fires
router.on(expr="$.outputs.score >= 0.7") >> high
router.on(expr="$.outputs.score >= 0.4 and $.outputs.score < 0.7") >> medium
router.on(expr="$.outputs.score < 0.4") >> low
```

**Edge Priority**: Higher priority edges are evaluated first, but ALL matching edges still fire. Use `priority` parameter: `node.on(expr="...", priority=10) >> handler`

## Setup

In [ ]:
from spark.nodes import Node, EdgeCondition
from spark.graphs import Graph
from spark.utils import arun
import time

## Example 1: Lambda Functions - Number Classification

In [ ]:
class NumberGenerator(Node):
    async def process(self, context):
        number = context.inputs.content.get('number', 42)
        print(f"🔢 Number: {number}")
        return {'number': number, 'timestamp': time.time()}

class LowNumberHandler(Node):
    async def process(self, context):
        number = context.inputs.content.get('number')
        print(f"📉 LOW: {number}")
        return {'category': 'low', 'message': f'{number} is less than 50'}

class HighNumberHandler(Node):
    async def process(self, context):
        number = context.inputs.content.get('number')
        print(f"📈 HIGH: {number}")
        return {'category': 'high', 'message': f'{number} is 50 or greater'}

# Create nodes and edges
generator = NumberGenerator()
low_handler = LowNumberHandler()
high_handler = HighNumberHandler()

generator.goto(low_handler, condition=EdgeCondition(lambda n: n.outputs.content.get('number', 0) < 50))
generator.goto(high_handler, condition=EdgeCondition(lambda n: n.outputs.content.get('number', 0) >= 50))

graph = Graph(start=generator)

print("=== Example 1: Lambda Conditional Routing ===")
for test in [25, 75, 50]:
    print(f"\nTest: {test}")
    result = await graph.run({'number': test})
    print(f"Result: {result.content['message']}")

**Key Points:**
- Lambda receives the node `n` as parameter
- Check `n.outputs.content` for the returned data
- Use `.get(key, default)` to avoid KeyError
- Mutually exclusive conditions (< 50 and >= 50) ensure only one path fires
- Check `n.outputs` (flows to next node), not `n.state` (node-local only)

## Example 2: Shorthand Routing - Intent Classification

In [ ]:
class IntentClassifier(Node):
    async def process(self, context):
        message = context.inputs.content.get('message', '')
        print(f"🤔 Classifying: '{message}'")
        
        message_lower = message.lower()
        if 'buy' in message_lower or 'purchase' in message_lower:
            intent = 'sales'
        elif 'help' in message_lower or 'problem' in message_lower:
            intent = 'support'
        elif 'refund' in message_lower or 'bill' in message_lower:
            intent = 'billing'
        else:
            intent = 'general'
        
        print(f"📋 Intent: {intent}")
        return {'message': message, 'intent': intent}

class SalesHandler(Node):
    async def process(self, context):
        print(f"💰 Sales team handling...")
        return {'department': 'sales', 'response': 'Our sales team will contact you!'}

class SupportHandler(Node):
    async def process(self, context):
        print(f"🔧 Support team handling...")
        return {'department': 'support', 'response': 'Our support team is on it!'}

class BillingHandler(Node):
    async def process(self, context):
        print(f"💳 Billing team handling...")
        return {'department': 'billing', 'response': 'Our billing team will review!'}

class GeneralHandler(Node):
    async def process(self, context):
        print(f"📞 General team handling...")
        return {'department': 'general', 'response': 'Thank you for contacting us!'}

# Create nodes and use shorthand routing
classifier = IntentClassifier()
classifier.on(intent='sales') >> SalesHandler()
classifier.on(intent='support') >> SupportHandler()
classifier.on(intent='billing') >> BillingHandler()
classifier.on(intent='general') >> GeneralHandler()

routing_graph = Graph(start=classifier)

print("=== Example 2: Shorthand Routing ===")
test_messages = [
    "I want to buy your product",
    "I need help with my device",
    "I was charged twice",
    "Hello!"
]

for message in test_messages:
    print(f"\n{'='*50}")
    result = await routing_graph.run({'message': message})
    print(f"✅ {result.content['response']}")

**Key Points:**
- `.on(key=value)` is shorthand for `EdgeCondition(equals={'key': 'value'})`
- Multiple keys use AND logic: `node.on(status='ready', count=10)` requires BOTH to match
- Most readable for simple equality checks
- Intents are mutually exclusive, so only one path fires

## Example 3: Multi-Level Decision Trees

In [ ]:
class UserAuthenticator(Node):
    async def process(self, context):
        user_id = context.inputs.content.get('user_id')
        is_authenticated = user_id is not None and user_id > 0
        print(f"🔐 Auth: user_id={user_id}, authenticated={is_authenticated}")
        return {'user_id': user_id, 'authenticated': is_authenticated}

class PermissionChecker(Node):
    async def process(self, context):
        user_id = context.inputs.content.get('user_id')
        is_admin = user_id < 10  # Admins have id < 10
        print(f"🔑 Permissions: admin={is_admin}")
        return {'user_id': user_id, 'authenticated': True, 'role': 'admin' if is_admin else 'user'}

class AdminDashboard(Node):
    async def process(self, context):
        print("👑 Admin Dashboard")
        return {'page': 'admin_dashboard', 'access': 'granted'}

class UserDashboard(Node):
    async def process(self, context):
        print("👤 User Dashboard")
        return {'page': 'user_dashboard', 'access': 'granted'}

class LoginPage(Node):
    async def process(self, context):
        print("🚫 Login Required")
        return {'page': 'login', 'access': 'denied'}

# Build decision tree
auth = UserAuthenticator()
permissions = PermissionChecker()
admin_dash = AdminDashboard()
user_dash = UserDashboard()
login = LoginPage()

# Level 1: auth check
auth.on(authenticated=True) >> permissions
auth.on(authenticated=False) >> login

# Level 2: role check
permissions.on(role='admin') >> admin_dash
permissions.on(role='user') >> user_dash

decision_tree = Graph(start=auth)

print("=== Example 3: Multi-Level Decision Tree ===")
for user_id, desc in [(5, 'Admin'), (100, 'User'), (None, 'Unauthenticated')]:
    print(f"\nTest: {desc}")
    result = await decision_tree.run({'user_id': user_id})
    print(f"✅ Page: {result.content['page']}, Access: {result.content['access']}")

**Key Points:**
- Each node can have multiple outgoing edges
- Downstream nodes can also branch, creating trees
- Use mutually exclusive conditions for single-path routing
- Use `priority` parameter if evaluation order matters
- For fan-out (multiple handlers), use overlapping conditions

## Example 4: Loops and Cycles - Quiz System

In [ ]:
class QuestionAsker(Node):
    async def process(self, context):
        attempt = context.state.get('attempt', 1)
        print(f"\n❓ Attempt {attempt}: What is 5 + 3?")
        
        # Simulate answers: wrong, wrong, correct
        simulated_answers = ['7', '9', '8']
        answer = simulated_answers[min(attempt - 1, 2)]
        print(f"Answer: {answer}")
        
        context.state['attempt'] = attempt + 1
        return {'answer': answer, 'attempt': attempt}

class AnswerChecker(Node):
    async def process(self, context):
        answer = context.inputs.content.get('answer')
        attempt = context.inputs.content.get('attempt')
        is_correct = answer == '8'
        
        print("✅ Correct!" if is_correct else f"❌ Wrong!")
        
        max_attempts = 3
        should_retry = not is_correct and attempt < max_attempts
        
        return {
            'is_correct': is_correct,
            'attempt': attempt,
            'should_retry': should_retry,
            'give_up': not is_correct and attempt >= max_attempts
        }

class SuccessHandler(Node):
    async def process(self, context):
        attempt = context.inputs.content.get('attempt')
        print(f"\n🎉 Success on attempt {attempt}!")
        return {'status': 'success', 'attempts': attempt}

class FailureHandler(Node):
    async def process(self, context):
        print(f"\n😔 Failed after 3 attempts.")
        return {'status': 'failed'}

# Build loop
asker = QuestionAsker()
checker = AnswerChecker()
success = SuccessHandler()
failure = FailureHandler()

asker >> checker
checker.on(is_correct=True) >> success
checker.on(should_retry=True) >> asker  # LOOP!
checker.on(give_up=True) >> failure

quiz_graph = Graph(start=asker)

print("=== Example 4: Loops and Cycles ===")
result = await quiz_graph.run()
print(f"\nFinal: {result.content}")

**Key Points:**
- Use `context.state` to track iteration count
- **Always define exit conditions** to prevent infinite loops
- Loops work the same as regular edges
- Common uses: retry logic, iterative refinement, conversational loops

## Example 5: Expression-Based Routing

In [ ]:
class ScoreCalculator(Node):
    async def process(self, context):
        correct = context.inputs.content.get('correct_answers', 0)
        total = context.inputs.content.get('total_questions', 10)
        score = (correct / total) * 100
        
        if score >= 90: grade = 'A'
        elif score >= 80: grade = 'B'
        elif score >= 70: grade = 'C'
        elif score >= 60: grade = 'D'
        else: grade = 'F'
        
        print(f"📊 Score: {score:.1f}% (Grade: {grade})")
        return {'score': score, 'grade': grade}

class ExcellentHandler(Node):
    async def process(self, context):
        print("🌟 Excellent! Grade A")
        return {'message': 'Excellent! Grade A'}

class GoodHandler(Node):
    async def process(self, context):
        print("👍 Good! Grade B or C")
        return {'message': 'Good! Grade B or C'}

class PassedHandler(Node):
    async def process(self, context):
        print("✅ Passed! Grade D")
        return {'message': 'Passed with Grade D'}

class FailedHandler(Node):
    async def process(self, context):
        print("📚 Failed. Grade F")
        return {'message': 'Failed. Grade F'}

# Create graph with expression-based routing
calculator = ScoreCalculator()
calculator.goto(ExcellentHandler(), condition=EdgeCondition(expr="$.outputs.grade == 'A'"))
calculator.goto(GoodHandler(), condition=EdgeCondition(lambda n: n.outputs.content.get('grade') in ['B', 'C']))
calculator.goto(PassedHandler(), condition=EdgeCondition(expr="$.outputs.grade == 'D'"))
calculator.goto(FailedHandler(), condition=EdgeCondition(expr="$.outputs.grade == 'F'"))

grading_graph = Graph(start=calculator)

print("=== Example 5: Expression-Based Routing ===")
for correct, desc in [(10, 'Perfect'), (8, 'Good'), (6, 'Passed'), (4, 'Failed')]:
    print(f"\nTest: {desc}")
    result = await grading_graph.run({'correct_answers': correct, 'total_questions': 10})
    print(f"Result: {result.content['message']}")

**Expression Syntax:**

```python
# Comparisons: ==, !=, >, <, >=, <=
node.on(expr="$.outputs.score >= 0.8") >> high

# Logical: and, or, not
node.on(expr="$.outputs.ready and $.outputs.score > 0.5") >> process

# Membership: in
node.on(expr="$.outputs.category in ['A', 'B', 'C']") >> premium

# Nested paths
node.on(expr="$.outputs.user.role == 'admin'") >> admin

# Boolean fields
node.on(expr="$.outputs.is_ready") >> ready_handler
```

**Use expressions for:**
- Declarative workflows (JSON configs)
- Complex conditions without lambda complexity
- Serializable routing logic

## Example 6: Complete Workflow - Content Moderation

In [ ]:
class ContentReceiver(Node):
    async def process(self, context):
        content = context.inputs.content.get('content', '')
        print(f"📥 Received: '{content[:50]}...'")
        return {'content': content, 'review_count': context.state.get('review_count', 0)}

class AutomaticFilter(Node):
    async def process(self, context):
        content = context.inputs.content.get('content', '').lower()
        banned_words = ['spam', 'scam', 'malware']
        has_banned = any(word in content for word in banned_words)
        
        if has_banned:
            print("🚫 Banned content detected")
            status = 'rejected'
        elif len(content) < 10:
            print("⚠️ Content too short")
            status = 'rejected'
        else:
            print("✅ Passed initial checks")
            status = 'needs_review'
        
        return {'content': context.inputs.content.get('content'), 'status': status}

class HumanReview(Node):
    async def process(self, context):
        content = context.inputs.content.get('content')
        review_count = context.inputs.content.get('review_count', 0)
        print(f"👤 Reviewer #{review_count + 1} checking...")
        
        content_lower = content.lower()
        if 'great' in content_lower or 'excellent' in content_lower:
            decision = 'approved'
        elif 'bad' in content_lower or 'terrible' in content_lower:
            decision = 'needs_revision'
        else:
            decision = 'approved'
        
        print(f"Decision: {decision}")
        context.state['review_count'] = review_count + 1
        
        return {
            'content': content,
            'status': decision,
            'review_count': review_count + 1,
            'needs_revision': decision == 'needs_revision' and review_count < 2
        }

class RevisionRequest(Node):
    async def process(self, context):
        print("📝 Requesting revision...")
        original = context.inputs.content.get('content')
        revised = original.replace('bad', 'great').replace('terrible', 'excellent')
        print(f"✏️ Revised content received")
        return {'content': revised, 'review_count': context.inputs.content.get('review_count', 0)}

class ApprovedHandler(Node):
    async def process(self, context):
        print("✅ Content published!")
        return {'status': 'published', 'message': 'Your content has been published'}

class RejectedHandler(Node):
    async def process(self, context):
        print(f"❌ Content rejected")
        return {'status': 'rejected', 'message': 'Your content violates our policies'}

# Build workflow
receiver = ContentReceiver()
filter_node = AutomaticFilter()
reviewer = HumanReview()
revision = RevisionRequest()
approved = ApprovedHandler()
rejected = RejectedHandler()

receiver >> filter_node
filter_node.on(status='rejected') >> rejected
filter_node.on(status='needs_review') >> reviewer
reviewer.on(status='approved') >> approved
reviewer.on(needs_revision=True) >> revision
revision >> receiver  # Loop back

moderation_workflow = Graph(start=receiver)

print("=== Example 6: Content Moderation Workflow ===")
test_cases = [
    {"content": "Check out this spam offer!", "description": "Banned"},
    {"content": "This is a great article about AI technology.", "description": "Good"},
    {"content": "This is a bad article.", "description": "Needs revision"}
]

for test in test_cases:
    print(f"\n{'='*60}\nTest: {test['description']}\n")
    result = await moderation_workflow.run(test)
    print(f"\n🎯 Status: {result.content.get('status')}")

**Workflow demonstrates:**
- Multiple routing strategies (shorthand + lambda)
- Multi-level decision trees
- Cycles (revision loop)
- State management (`review_count`)
- State vs Outputs: `context.state` is node-local; use `outputs` to share data between nodes

```
ContentReceiver → AutomaticFilter ─┬→ Rejected
      ↑                             └→ HumanReview ─┬→ Approved
      │                                             ├→ RevisionRequest
      └─────────────────────────────────────────────┘  (loop)
```

## Quick Reference

### Three Routing Methods

```python
# 1. Lambda (flexible)
node.goto(next_node, condition=EdgeCondition(lambda n: n.outputs.content.get('score') > 80))

# 2. Shorthand (readable, AND logic)
node.on(status='active', count=10) >> handler  # Both must match

# 3. Expression (declarative)
node.on(expr="$.outputs.score > 0.8 and $.outputs.ready") >> handler
```

### Fan-Out vs Single-Path

```python
# Fan-Out: Both fire if score=0.8
node.on(expr="$.outputs.score > 0.7") >> logger
node.on(expr="$.outputs.score > 0.5") >> validator

# Single-Path: Mutually exclusive
node.on(expr="$.outputs.score >= 0.7") >> high
node.on(expr="$.outputs.score >= 0.4 and $.outputs.score < 0.7") >> medium
node.on(expr="$.outputs.score < 0.4") >> low
```

### Common Patterns

```python
# Simple branching
classifier.on(intent='sales') >> sales_handler

# Retry loop
processor >> checker
checker.on(success=False) >> processor  # Loop
checker.on(success=True) >> next_step   # Exit

# Priority
node.on(expr="$.outputs.critical", priority=100) >> critical_handler
```

### Best Practices

1. Use shorthands for simple equality
2. Use expressions for comparisons and logic
3. Use lambdas for complex custom logic
4. Mutually exclusive conditions = single path
5. Always define exit conditions for loops
6. Use `context.state` to track iterations (node-local)
7. Use outputs to share data between nodes
8. Provide defaults: `.get(key, default)`

## Exercises

### Exercise 1: Temperature Router

Create a temperature classification system:
- Cold: < 50°F → ColdHandler
- Mild: 50-75°F → MildHandler  
- Hot: > 75°F → HotHandler

Use shorthand routing.

In [ ]:
# Exercise 1: Your code here

class TemperatureClassifier(Node):
    async def process(self, context):
        # TODO: Implement classification
        pass

# TODO: Create handlers and routing

### Exercise 2: Retry Loop

Create a workflow that:
1. Attempts an operation (random success/failure)
2. Retries up to 3 times on failure
3. Routes to success/failure handlers

Use `context.state` to track attempts.

In [ ]:
# Exercise 2: Your code here
import random

class OperationNode(Node):
    async def process(self, context):
        # TODO: Implement with retry logic
        pass

## Solutions

In [ ]:
# Solution 1: Temperature Classification

class TemperatureClassifier(Node):
    async def process(self, context):
        temp = context.inputs.content.get('temperature', 70)
        print(f"🌡️ Temperature: {temp}°F")
        
        if temp < 50: category = 'cold'
        elif temp <= 75: category = 'mild'
        else: category = 'hot'
        
        return {'temperature': temp, 'category': category}

class ColdHandler(Node):
    async def process(self, context):
        temp = context.inputs.content.get('temperature')
        print(f"❄️ Cold: {temp}°F - Bundle up!")
        return {'advice': 'Wear a heavy coat!'}

class MildHandler(Node):
    async def process(self, context):
        temp = context.inputs.content.get('temperature')
        print(f"🌤️ Mild: {temp}°F - Perfect!")
        return {'advice': 'Light jacket is fine!'}

class HotHandler(Node):
    async def process(self, context):
        temp = context.inputs.content.get('temperature')
        print(f"🔥 Hot: {temp}°F - Stay cool!")
        return {'advice': 'Shorts and sunscreen!'}

temp_classifier = TemperatureClassifier()
temp_classifier.on(category='cold') >> ColdHandler()
temp_classifier.on(category='mild') >> MildHandler()
temp_classifier.on(category='hot') >> HotHandler()

temp_graph = Graph(start=temp_classifier)

print("=== Solution 1 ===")
for temp in [30, 65, 85]:
    print(f"\nTest: {temp}°F")
    result = await temp_graph.run({'temperature': temp})
    print(f"✅ {result.content['advice']}")

In [ ]:
# Solution 2: Retry Loop
import random

class OperationNode(Node):
    async def process(self, context):
        retry_count = context.state.get('retry_count', 0)
        print(f"\n🔄 Attempt #{retry_count + 1}")
        
        success = random.random() < 0.4  # 40% success rate
        print("✅ Success!" if success else "❌ Failed!")
        
        context.state['retry_count'] = retry_count + 1
        return {'success': success, 'retry_count': retry_count + 1}

class CheckerNode(Node):
    async def process(self, context):
        success = context.inputs.content.get('success')
        retry_count = context.inputs.content.get('retry_count')
        
        return {
            'success': success,
            'retry_count': retry_count,
            'should_retry': not success and retry_count < 3,
            'give_up': not success and retry_count >= 3
        }

class SuccessHandler(Node):
    async def process(self, context):
        count = context.inputs.content.get('retry_count')
        print(f"\n🎉 Success after {count} attempt(s)!")
        return {'status': 'completed', 'attempts': count}

class FailureHandler(Node):
    async def process(self, context):
        print(f"\n😔 Failed after 3 attempts.")
        return {'status': 'failed', 'attempts': 3}

operation = OperationNode()
checker = CheckerNode()

operation >> checker
checker.on(success=True) >> SuccessHandler()
checker.on(should_retry=True) >> operation
checker.on(give_up=True) >> FailureHandler()

retry_graph = Graph(start=operation)

print("=== Solution 2 ===")
result = await retry_graph.run()
print(f"\nFinal: {result.content}")

## Summary

### You've Learned:

✅ Three ways to define conditions (lambdas, shorthands, expressions)  
✅ Fan-out routing (ALL matching edges fire)  
✅ Single-path routing (mutually exclusive conditions)  
✅ Multi-level decision trees  
✅ Loops and cycles with exit conditions  
✅ State management for iterations

### Key Takeaways:

- Use **shorthands** for readability
- Use **expressions** for declarative logic
- Use **lambdas** for complex conditions
- **Mutually exclusive** conditions for single-path
- **Always define exit conditions** for loops
- Use **`context.state`** for node-local data
- Use **outputs** to share data between nodes

### Next Steps:

**Tutorial 5**: LLM-Powered Routing  
Learn to use LLMs for intelligent routing decisions and build ReAct-style reasoning agents.

**Related Examples**: `e004_simple_flow_graph.py`, `e005_simple_flow_using_shorthands.py`

---

You're now ready to build intelligent, adaptive workflows! 🚀